# 环境介绍
本示例运行在google colab L4，如果选择T4可能会遇到资源不足的情况。

# 本示例概要
本示例主要是带领大家如何使用Unsloth对已经将DeepSeek-R1蒸馏到小模型Llama8b的版本进行微调所构建医疗专家的模型，那我们接着来实作吧。

# 利用 Unsloth 微調 DeepSeek-R1-Distill-Llama-8B

在此筆記本中，它將示範如何使用醫療資料集透過 Unsloth 微調 DeepSeek-R1-Distill-Llama-8B 實操。

## 為什麼需要LLM微調？

微調可以使模型在特定任務上有更好的表現，使其在實際應用中更有效、更靈活。像這次示例就是想構建醫療專家模型，希望回答的內容更加貼近目標答案。
此流程對於根據特定任務或領域自訂現有模型至關重要。

### 安裝unsloth
unslotg 是開源大模型微調工具包

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install bitsandbytes unsloth_zoo

In [ ]:
!pip install -q openai-whisper
!pip install -q ffmpeg

## 選擇一個基礎模型

1. 選擇符合你用例的模型(例如視覺需求就選擇視覺表現好的模型作為基座，如果是針對編碼表現好的就選擇coder的模型作為基座)
2. 評估您的儲存、運算能力和資料集（查看自己的硬體和手頭所擁有的數據集，來選擇一款合適的參數規格模型）
3. 選擇模型和參數
4. 在基礎模型和指導模型之間進行選擇

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
# from_pretrained for download model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

## 微調前的推理

In [4]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>{}"""

In [5]:
question = "一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？"


FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
好的，我现在需要帮助处理一个急性阑尾炎患者的情况。患者已经发病5天，腹痛有所减轻但仍然发热。在体检时，发现了右下腹有压痛的包块。作为医生，我需要仔细思考如何处理这个情况。

首先，急性阑尾炎通常表现为右下腹疼痛、发热、发凉、腹部压痛等症状。患者已经有5天的病程，虽然腹痛有所缓解，但仍有发热，这可能意味着炎症尚未完全缓解，或者可能有其他并发症需要考虑。

体检发现右下腹有压痛的包块，这可能有两种解释：一是阑尾炎的包块，尤其是右下腹的压痛，这与阑尾炎的典型表现一致；二是其他如腺体炎症或其他胰腺、肝脏的问题，但考虑到发热和腹痛，阑尾炎仍然是首选。

接下来，我需要评估患者的症状和体征是否需要进一步的影像学检查。X光或超声检查可以帮助确认包块的性质，是否有扭转、积液或其他结构异常。例如，如果包块是扭转的阑尾，可能需要手术治疗；如果是积液，可能需要引流。

如果包块是阑尾的扭转或阻塞，可能需要进行手术，特别是如果患者有明显的症状，如持续发热、恶心、呕吐等。手术可以帮助缓解症状，防止感染。

另一方面，如果包块不是阑尾的，而是其他结构，比如胰腺炎症或肝脏问题，可能需要针对性治疗，比如抗生素、抗病毒药物或其他内科治疗。

此外，患者的腹痛虽然有所缓解，但仍有发热，这可能提示炎症仍在活动，需要继续抗生素治疗以防止感染。同时，评估是否有其他症状，如发凉、腹泻等，以帮助诊断。

在处理过程中，应该密切监测患者的症状，及时进行影像学检查，以明确包块的性质和后续治疗方案。同时，考虑患者的年龄、病史和其他可能的并发症，制定个性化的治疗计划。

总结来说，处理这种情况的关键步骤是：1. 确认诊断，2. 影像学检查确认包块性质，3. 根据检查结果决定是否手术或其他治疗，4. 继续观察和管理患者的症状。

现在，我需要将这些思考整理成一个连贯的步骤，确保每一步都清晰明确，能够指导实际的处理过程。
</think>

对于一个患有急性阑尾炎的患者，腹痛虽有缓解但仍发热，体检发现右下腹压痛包块，处理步骤如下：

1. **确认诊断**：基于症状和体征，确认为急性阑尾炎，排除其他可能如腺体炎症。

2. **影像学检查**：进行X光或超声，确定包块性质，如扭转、积液或其他异常。

3. **评估并发症**：检查是否有扭转或阻塞，决定是否需要手术。

4. **继续抗生素治疗**：根据检

## 準備資料集

將使用醫療資料集 [https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/](https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/) 來訓練所選模型。

In [6]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

### 重要通知

在每個訓練資料集條目末尾添加 EOS（序列結束）標記至關重要，否則您可能會遇到無限的生成。

In [7]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [ ]:
from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", 'zh', split = "train[0:500]", trust_remote_code=True)
print(dataset.column_names)

為了讓「Ollama」和「llama.cpp」像自訂「ChatGPT」聊天機器人一樣運行，我們必須只有 2 列——「指令」和「輸出」列。我們需要將資料集轉換為適當的結構。

In [9]:
dataset = dataset.map(formatting_prompts_func, batched = True)
dataset["text"][0]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

'Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本

訓練模型
現在讓我們使用 Huggingface TRL 的 `SFTTrainer`。

我們使用peft使用變量，peft是參數高效的微調,接著用SFTTrainer創建訓練器，同時也希望train loss值越低越好。接著一樣的問題進行問答，來查看跟數據集中的答案是否接近與滿足需求，反覆調參。





In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.275200
2,2.349100
3,2.296400
4,2.167300
5,2.077000
6,2.044300
7,2.155000
8,1.924400
9,1.985100
10,1.879300


## 微調後的推理

我們再用同樣的問題來推理，看看有什麼不同。

In [13]:
print(question)

一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？


In [14]:
FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
这个病人已经有五天的急性阑尾炎了，虽然他的腹痛稍微好了一点，但他还是发烧的。听说在右下腹部有个包块，而且有压痛。嗯，这个包块听起来像是阑尾炎的典型表现。既然他已经有五天了，应该已经到达急性阶段了。

我想到，急性阑尾炎通常是需要抗生素来治疗的。因为可能有感染，抗生素可以帮助控制细菌感染。所以，首先要考虑给他开个抗生素的药物来处理。

不过，还要考虑到他的包块和压痛，这可能意味着他已经有了一个坏死的阑尾，或者可能有阻塞的情况。这种情况下，可能需要进行手术来处理。

但我也要考虑到他的状况可能不太稳定。因为他已经发烧了五天，可能已经有感染的风险增加，手术可能会增加感染的风险。

嗯，综合考虑，我觉得在这段时间内，最好的办法是先用抗生素来控制感染。这样可以降低感染风险，然后在病情稳定之后再考虑手术。

所以，首先给他开抗生素，控制感染，然后再观察病情的变化，决定是否需要手术。
</think>
对于一个患有急性阑尾炎的病人，已经发病5天，腹痛减轻但仍然发热，且在右下腹部发现压痛的包块，建议的处理步骤如下：

1. **抗生素治疗**：首先，开具抗生素以控制感染。通常选择针对细菌感染的抗生素，如青霉素、头孢素或第三代头孢菌素等，以控制感染。

2. **监测病情**：观察病人的病情变化，包括腹痛的程度、发热情况、包块的变化以及是否有新的症状出现。这些信息有助于评估病情的稳定性和进一步的处理策略。

3. **评估是否需要手术**：如果在抗生素治疗的同时，病人的包块大小不减小或有明显的增大，或者出现严重的腹痛、发热、恶寒、呕吐等症状，可能需要进一步评估是否需要手术。

4. **手术评估**：如果在抗生素治疗下病情不改善，且包块增大或有阻塞，建议在病情稳定后进行阑尾切除术，以排除阑尾炎。

5. **术后护理**：如果决定进行手术，术后需要密切观察，控制感染风险，并根据术后病情进行适当的治疗。

总之，建议先进行抗生素治疗以控制感染，随后根据病情的变化决定是否进行手术。<｜end▁of▁sentence｜>


## 上傳模型至 HuggingFace

現在，讓我們儲存微調後的型號並將其上傳到 HuggingFace。

### 將微調後的模型儲存為GGUF格式

將對應的“if”設為“True”，選擇我們喜歡的llama.cpp的GGUF格式。

HUGGINGFACE_TOKEN請輸入你在HUGGINGFACE的TOKEN。

In [ ]:
from google.colab import userdata

HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

In [ ]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

### 將模型推送到 HuggingFace

如果您還沒有這樣做，請為您的模型建立一個模型類型儲存庫。

In [ ]:
from huggingface_hub import create_repo
create_repo("joyce914/medical-model", token=HUGGINGFACE_TOKEN, exist_ok=True)

In [ ]:
model.push_to_hub_gguf("joyce914/medical-model", tokenizer, token = HUGGINGFACE_TOKEN)

### Ollama 運行 HuggingFace 模型

```bash
ollama run hf.co/{username}/{repository}:{quantization}
```

### Ollama inference

```bash
curl http://localhost:11434/api/chat -d '{ \
  "model": "", \
  "messages": [ \
    { "role": "user", "content": "一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？" } \
  }'
```